In [20]:
import numpy as np
import cv2
import time
from IPython.display import clear_output
from tkinter import *

%run math_calibration.ipynb

def get_available_cameras (upper_bound = 10, lower_bound = 0):
    available = []
    
    for i in range (lower_bound, upper_bound):
        cap = cv2.VideoCapture (i)
    
        if (cap.isOpened ()):
            available.append (i)
    
        cap.release ()
    
    return available

def put_text (img, text, str_num):
    font                   = cv2.FONT_ITALIC
    bottomLeftCornerOfText = (10, 50 + 30 * str_num)
    fontScale              = 0.8
    fontColor              = (15, 15, 215)
    lineType               = 2
    
    cv2.putText(img, text, 
        bottomLeftCornerOfText, 
        font, 
        fontScale,
        fontColor,
        lineType)

#TODO: implement class, incapsulating input source
#possible inputs: video, camera, photo

def output_calib_data (calib_data, frame, str_num):
    fr = frame.copy ()
    
    if (calib_data is not None and calib_data != 0):
        for key in calib_data.keys ():
            if (key == "opt_len" or
                key == "optical"):
                continue

            if (key == "corners"):
                for i in range (3):
                    x = calib_data [key] [i, 0]
                    y = calib_data [key] [i, 1]

                    cv2.circle (fr, (int (x), int (y)), 7, (123, 234, 145), -1)

                continue

            #print (key + ": ", calib_data [key])
            put_text (fr, key + ": " + str (np.around (calib_data [key], 1)), str_num)
            str_num += 1
    
    return fr

CAMERA = 0
VIDEO  = 1
PHOTO  = 2

video_path = ""
video_file = ""

photo_path = "/Users/elijah/Dropbox/Programming/RoboCup/nao_cv/floor_desk/"
photo_file = "IMAGE_0_0_35.jpg"

output_path = "/Users/elijah/Dropbox/Programming/RoboCup/nao_cv/geometrical/chessboard_images/"

ROTATED_FRAME = True

def main ():
    INPUT_SOURCE = CAMERA

    cam_num = max (get_available_cameras ())

    cam = cv2.VideoCapture (cam_num)

    if (INPUT_SOURCE != CAMERA):
        cam.release ()

    if (INPUT_SOURCE == VIDEO):
        cam = cv2.VideoCapture (video_path + video_file)

    elif (INPUT_SOURCE == PHOTO):
        img = cv2.imread (photo_path + photo_file)

    #cv2.namedWindow ("frame", cv2.WINDOW_NORMAL)
    cv2.namedWindow ("frame")
    
    #cv2.resizeWindow ("frame", (640*2, 480*2))
    cv2.resizeWindow ("frame", (480, 640))

    str_num = 0

    calib_data = 0
    
    IMAGES_NUM = 10
    begin_timestamp = 0
    img_writing_mode = False
    written = 0
    
    #положение шарнира камеры относительно ковра: x = 30 - x0; y = 62; z = 86
    
    #x00 = 11 сантиметров
    #y00 = 0 сантиметров
    
    #185 17 35 54
    #131 22
    
    x     = ""
    y     = "100"
    angle = ""

    unique_str = angle + "_" + x + "_" + y + "_"
    
    while (True):
        if (INPUT_SOURCE == CAMERA or INPUT_SOURCE == VIDEO):
            ret, frame_ = cam.read ()

        elif (INPUT_SOURCE == PHOTO):
            frame_ = img.copy ()

        if (ROTATED_FRAME == False):
            frame = frame_
        
        else:
            sh = frame_.shape
            #frame = frame_.reshape ((sh [1], sh [0], sh [2]))
            frame = frame_.transpose ((1, 0, 2))
            frame = np.flip (frame, 1)
            #frame = frame [:] [ : : -1] [:]
        
        str_num = 0

        cv2.waitKey (1)    

        sz = 7
        shape = (sz, sz)
        
        calib_data_ = calibration_data (frame, shape, 38)
        
        if (img_writing_mode == True):
            if (written == IMAGES_NUM):
                img_writing_mode = False
                written = 0
            
            else:
                cv2.imwrite (output_path + unique_str + str (begin_timestamp) + str (written) + ".jpg", frame)
                written += 1
                
                put_text (frame, "writing " + str (written) + "/" + str (IMAGES_NUM), str_num)
                str_num += 1

        if (calib_data_ is not None):
            calib_data = calib_data_

        frame = output_calib_data (calib_data, frame, str_num)

        #frame__ = cv2.resize (frame, (640*2, 480*2))

        frame__ = cv2.resize (frame, (480, 640))
        
        cv2.imshow ("frame", frame)

        time.sleep (0.02)

        clear_output (wait=True)
        
        keyb = cv2.waitKey (1) & 0xFF
        
        if (keyb == ord('f')):
            if (time.time () - begin_timestamp > 1.0):
                
                
                begin_timestamp = time.time ()
                img_writing_mode = True

        if (keyb == ord('q')):
            break

    cam.release ()

    cv2.destroyAllWindows()

if __name__ == "__main__":
    main ()

[[130. 134.]
 [150.  18.]
 [359. 133.]]
shape:  [480. 640.   0.]
pixels: 
[[ 110. -186.   -0.]
 [  90. -302.   -0.]
 [-119. -187.   -0.]]

real coords: 
[[-19.   0.   0.]
 [-19.  38.   0.]
 [ 19.   0.   0.]]


от края ковра 53 сантиметра

In [6]:
print (cam_num)

NameError: name 'cam_num' is not defined

In [5]:
#arr = np.array ([[1]], np.uint8)
#print (cast (arr))

path = "/Users/elijah/Dropbox/Programming/RoboCup/nao_cv/floor_desk/"
file = "IMAGE_0_0_35.jpg"

img = cv2.imread (path + file)

_, corners = cv2.findChessboardCorners (img, (3, 3))

print (corners)

print (cast (corners))

[[[195.58002 175.85455]]

 [[236.79152 175.2833 ]]

 [[277.93045 174.5153 ]]

 [[318.5652  173.04854]]

 [[359.58334 173.62636]]

 [[400.54987 174.79799]]

 [[441.56024 174.55977]]

 [[189.33018 197.31569]]

 [[233.0029  196.33333]]

 [[276.3815  195.30241]]

 [[319.5183  194.4003 ]]

 [[362.5954  195.45412]]

 [[405.79147 196.00374]]

 [[449.17004 196.61412]]

 [[182.24664 221.93114]]

 [[228.50356 221.45505]]

 [[274.5053  220.48799]]

 [[320.43356 218.2249 ]]

 [[366.18097 220.28436]]

 [[411.8309  220.60376]]

 [[457.69266 220.86298]]

 [[174.30031 249.10228]]

 [[223.54288 248.47144]]

 [[272.68268 248.34909]]

 [[321.3567  245.4806 ]]

 [[370.02307 247.69383]]

 [[418.5618  248.49545]]

 [[467.81277 248.11949]]

 [[165.18752 280.6722 ]]

 [[218.27158 279.86685]]

 [[270.35815 278.8809 ]]

 [[322.47955 277.30646]]

 [[374.41776 278.45273]]

 [[426.41663 279.71588]]

 [[478.8296  279.62265]]

 [[154.29143 316.3992 ]]

 [[211.29738 315.12518]]

 [[267.5168  312.38098]]

 [[323.61203